In [141]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input , LSTM , Lambda , RepeatVector , TimeDistributed , Layer , Embedding , Dense
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K

In [142]:
sample_sentences = [
    "AI is transforming the world.",
    "DL model requires large data sets.",
    "NLP enables chatbots.",
    "Computer vision helps in image recommendation.",
    "Generative models create realistic content.",
    "Machine learning powers recommendation engines.",
    "AI is revolutionizing healthcare diagnostics.",
    "Deep learning mimics human brain functions.",
    "Natural language processing boosts search accuracy.",
    "Reinforcement learning optimizes robotic movement.",
    "Supervised learning requires labeled data.",
    "Unsupervised models discover hidden patterns.",
    "Neural networks are inspired by the human brain.",
    "AI improves customer service through automation.",
    "Transfer learning speeds up model training.",
    "CNNs are used in image classification.",
    "RNNs are ideal for sequential data.",
    "Transformers are the backbone of modern NLP.",
    "GANs generate realistic fake images.",
    "AI assists in medical image diagnosis.",
    "ML algorithms detect fraud in transactions.",
    "AI is used in autonomous vehicle systems.",
    "Speech recognition converts voice to text.",
    "NLP enables sentiment analysis on reviews.",
    "AI chatbots handle customer queries efficiently.",
    "AI-based systems enhance supply chain efficiency.",
    "Computer vision helps in quality control.",
    "AI detects anomalies in manufacturing.",
    "Predictive analytics forecasts future trends.",
    "Big data fuels machine learning models.",
    "AI models classify email as spam or not.",
    "Self-driving cars rely on deep learning.",
    "AI models predict stock market trends.",
    "AI in agriculture detects crop diseases.",
    "Edge AI enables on-device predictions.",
    "AI improves personalization in e-commerce.",
    "AI tools enhance cybersecurity defenses.",
    "AI helps detect deepfakes in media.",
    "AI is applied in facial recognition systems.",
    "AI models track user behavior online.",
    "AI translates text between languages.",
    "AI in finance helps assess credit risk.",
    "AI systems generate music and art.",
    "AI helps automate repetitive tasks.",
    "ML helps in churn prediction for companies.",
    "Reinforcement learning trains game agents.",
    "AI in logistics optimizes delivery routes.",
    "AI is transforming the legal industry.",
    "AI helps in drug discovery research.",
    "Robotics combined with AI improves manufacturing.",
    "ML models help in market segmentation.",
    "AI detects network intrusions.",
    "AI systems recommend movies and shows.",
    "AI chatbots simulate human conversation.",
    "Voice assistants use NLP for commands.",
    "AI enables hands-free smart devices.",
    "Image recognition identifies objects and faces.",
    "AI personalizes online learning experiences.",
    "AI powers virtual teaching assistants.",
    "ML improves energy usage predictions.",
    "AI in retail optimizes inventory management.",
    "AI is reshaping the entertainment industry.",
    "AI algorithms detect product defects.",
    "AI improves quality control in factories.",
    "AI accelerates data labeling processes.",
    "AI helps in climate modeling and forecasting.",
    "ML in insurance assesses claim risks.",
    "AI enhances fraud detection mechanisms.",
    "AI reads and summarizes long documents.",
    "AI helps generate synthetic data.",
    "ML models predict customer lifetime value.",
    "AI is used in smart home automation.",
    "AI supports emotion detection from images.",
    "AI helps in content moderation on social platforms.",
    "AI systems understand natural language queries.",
    "AI enables smart search recommendations.",
    "ML improves dynamic pricing strategies.",
    "AI predicts customer purchasing behavior.",
    "AI enhances AR and VR experiences.",
    "AI supports real-time translation services.",
    "AI identifies trends from social media.",
    "AI is used in handwriting recognition.",
    "AI assists with document classification.",
    "ML aids in target marketing campaigns.",
    "AI helps automate resume screening.",
    "AI bots aid in virtual interviews.",
    "AI supports accessibility tools for the disabled.",
    "AI monitors worker safety in industries.",
    "ML models detect disease outbreaks early.",
    "AI powers interactive learning apps.",
    "AI improves video surveillance analytics.",
    "AI helps optimize ad placement strategies.",
    "ML is used in predictive text typing.",
    "AI assists in drone navigation systems.",
    "AI enables emotion-aware applications.",
    "ML helps optimize sales forecasts.",
    "AI drives intelligent automation tools.",
    "AI speeds up document verification.",
    "AI helps with smart farming practices.",
    "ML improves personalized product recommendations.",
    "AI tools power automated translation.",
    "AI enhances call center efficiency.",
    "AI provides insights from large datasets.",
    "AI is reshaping how businesses operate.",
    "ML helps predict housing prices.",
    "AI systems automate",
]

In [143]:
from typing import Sequence
#Tokenizer
vocab_list = 100
Sequence_length = 10
text_processor = Tokenizer(num_words=vocab_list, oov_token="<OOV>")
text_processor.fit_on_texts(sample_sentences)
tokenized = text_processor.texts_to_sequences(sample_sentences)
padded_sequences = pad_sequences(tokenized, maxlen=Sequence_length, padding="post")

In [144]:
#Model Config
embed_dim = 64
latent_space = 16

In [145]:
#Encoder
input_layer = Input(shape=(Sequence_length,),name="text_input")
embed_layer = Embedding(input_dim=vocab_list, output_dim=embed_dim,mask_zero=True)(input_layer)
lstm_encoded = LSTM(64)(embed_layer)

In [146]:
z_mu = Dense(latent_space, name="latent_mean")(lstm_encoded)
z_log_sigma = Dense(latent_space, name="latent_log_var")(lstm_encoded)

In [147]:
def sample_data(args):
    mu, log_sigma = args
    eps = K.random_normal(shape=K.shape(mu))
    return mu + K.exp(log_sigma) * eps

In [148]:
Z_vector = Lambda(sample_data , name='latent_sampler')([z_mu,z_log_sigma])

In [149]:
#KL Divergence as Layer
class KLLoss(Layer):
  def call (self,inputs):
    mu, log_sigma = inputs
    kl = -0.5 * K.sum(1 + log_sigma - K.square(mu) - K.exp(log_sigma),axis=1)
    self.add_loss(K.mean(kl))
    return inputs

In [150]:
Kl_triggers =KLLoss()([z_mu,z_log_sigma]) #Activation Kl loss

In [151]:
#Decoder
repeat_z = RepeatVector(Sequence_length)(Z_vector)
decoded_lstm = LSTM(64,return_sequences=True)(repeat_z)
decoded_output = TimeDistributed(Dense(vocab_list,activation='softmax'))(decoded_lstm)

In [152]:
#Full VAE Model
vae_model = Model(input_layer,decoded_output,name="text_vae_model")
vae_model.compile(optimizer="adam",loss="sparse_categorical_crossentropy")

In [153]:
#Prepare output for training
target_output = np.expand_dims(padded_sequences,-1)
vae_model.fit(padded_sequences,target_output,epochs=50,batch_size=2)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 4.4508
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.4932
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.1204
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.0449
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.9723
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.6908
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.7582
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.6709
Epoch 9/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 1.6417
Epoch 10/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 1.6666
Epoch 11/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.6978
Epoch 12/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.5743
Epoch 13/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.6127
Epoch 14/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.6184
Epoch 15/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.5780
Epoc

In [154]:
#encoder model
text_encoder = Model(input_layer,z_mu,name="text_encoder")

In [155]:
print(target_output)

[[[ 2]
  [ 6]
  [42]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[95]
  [43]
  [44]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[16]
  [13]
  [26]
  ...
  [ 0]
  [ 0]
  [ 0]]

 ...

 [[ 2]
  [ 6]
  [84]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[ 5]
  [ 4]
  [36]
  ...
  [ 0]
  [ 0]
  [ 0]]

 [[ 2]
  [10]
  [37]
  ...
  [ 0]
  [ 0]
  [ 0]]]


In [156]:
print(text_encoder)

<Functional name=text_encoder, built=True>


In [157]:
#Decoder Model
latent_input = Input(shape=(latent_space,) , name="decoder_input")
repeated_input = RepeatVector(Sequence_length)(latent_input)
decoder_seq = LSTM(64,return_sequences=True)(repeated_input)
final_output = TimeDistributed(Dense(vocab_list,activation='softmax'))(decoder_seq)
text_decoder = Model(latent_input,final_output,name="decoder_only")

In [158]:
import numpy as np
random_z = np.random.normal(size=(1,latent_space))
generated_sequence_probs = text_decoder.predict(random_z)
generated_sequence_ids = np.argmax(generated_sequence_probs,axis=-1)[0]
index_to_word = {v: k for k , v in text_processor.word_index.items()}
index_to_word[0] = ' '
index_to_word[text_processor.word_index.get('<OOV>')] = '<OOV>'
generated_words = [index_to_word.get(idx,"<UNK>") for idx in generated_sequence_ids]
generated_text = ' '.join(generated_words).strip()
print("Generated Text :\n",generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
Generated Text :
 detection detection detection learning learning learning learning learning learning learning


In [159]:
def temp_sampling(probability,temp = 1.0):
    a = np.log(probability+1e-9)/temp
    b = np.exp(a)
    c = b/np.sum(b)
    return np.random.choice(len(c),p=c)

temp = 0.5
random_z = np.random.normal(size=(1,latent_space))
generated_sequence_probs = text_decoder.predict(random_z)[0]
generated_sequence_ids = [temp_sampling(words,temp) for words in generated_sequence_probs]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


In [140]:
#convert idx to words
index_to_word = text_processor.index_word
index_to_word[0] = " "
generated_words = [index_to_word.get(idx,"<UNK>") for idx in generated_sequence_ids]
generated_text = ' '.join(generated_words).strip()
print("Generated Text :\n",generated_text)


Generated Text :
 in from enables on fraud improves queries large from is


In [ ]:
#Image generator
#Encoder
#Decoder
#loss
#NLP

In [160]:
#Questions
#1. What is Tokenizer
#2. What is Embedding
#3. What is Stop word
#4. NLU
#5. NLZ
#6. What is bag of words
#7. What is stemming
#8. What is IDF
#9. What is POS tagging
#10. Explain Embedding Techniques
#11. Count Vectorizer
#12. diff between count and tfidf vectorizer
#13. what is Transformer and how it works '
#14. what is VERT model
#15. How Woulld you handle out of vocabulary words in text corpus
#16. What is masked language model (MLM)
#17. what is NMT (Neural machine translator)
#18. Pre-trained NMT model : helsiki-opus-mt-x-yy , Facebook/WMT-19-en-de , T5-base/T5-Large , Facebook/MLLB-200-distilled-600M
